In [1]:
% run 1-datasource.ipynb

In [2]:
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

import torch.nn.functional as F
from skorch.net import NeuralNetClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
scaler_1 = StandardScaler()
scaler_2 = StandardScaler()

In [4]:
for _, i in icebergs.iterrows():
    scaler_1.partial_fit(i.band_1.reshape(1, -1))
    scaler_2.partial_fit(i.band_2.reshape(1, -1))

In [5]:
y = icebergs.is_iceberg.values.astype(np.float32)
x = (
    np.stack(
        [
            scaler_1.transform(np.stack(icebergs.band_1)),
            scaler_2.transform(np.stack(icebergs.band_2)),
        ],
        axis=1
    ).reshape(-1, 2, 75, 75)
).astype(np.float32)

In [6]:
x.dtype, y.dtype

(dtype('float32'), dtype('float32'))

In [7]:
type(x), type(y)

(numpy.ndarray, numpy.ndarray)

In [8]:
y.shape, x.shape

((1604,), (1604, 2, 75, 75))

In [9]:
insize1 = 2 
outsize1 = 32 
outsize2 = 32
outsize3 = 32 
outsize4 = 32 

class Net(nn.Module):
    def __init__(self,outsize1=32,outsize2=32):
        super().__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(insize1, outsize1, kernel_size=5, stride=1, padding=2, groups=2),
            nn.BatchNorm2d(outsize1),
            nn.ReLU(),
            nn.MaxPool2d(4)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(outsize1, outsize2, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(outsize2, outsize3, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(outsize3, outsize4, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,1)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [10]:
#redifining scoring for sklearn 
from skorch import NeuralNet
from skorch.utils import to_numpy
from sklearn.metrics import log_loss
class NNplusplus(NeuralNet):
    def score(self,X,target):
        y_preds = []
        for yp in self.forward_iter(X, training=False):
            y_preds.append(to_numpy(yp.sigmoid()))
       
        y_preds = np.concatenate(y_preds, 0)
   
        return log_loss(target,y_preds)

In [11]:
model = NNplusplus(
    Net,
    optimizer=torch.optim.Adam,
    criterion=nn.BCEWithLogitsLoss, #regularization
    max_epochs=250,
    batch_size=25,
    lr=0.000001,
    use_cuda = True
)

In [ ]:
x.dtype, y.dtype

(dtype('float32'), dtype('float32'))

In [ ]:
model.fit(x,y)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.6976        0.7141  1.9319
      2        0.6947        0.7175  0.9357
      3        0.6921        0.7136  0.8975
      4        0.6896        0.7100  0.8703
      5        0.6872        0.7066  0.8663
      6        0.6850        0.7034  0.8654
      7        0.6830        0.7004  0.8704
      8        0.6811        0.6975  0.8659
      9        0.6792        0.6949  0.8659
     10        0.6775        0.6925  0.8940
     11        0.6759        0.6901  0.8681
     12        0.6743        0.6880  0.8694
     13        0.6729        0.6859  0.8686
     14        0.6714        0.6839  0.8792
     15        0.6701        0.6821  0.8687
     16        0.6688        0.6802  0.8696
     17        0.6675        0.6785  0.8661
     18        0.6663        0.6769  0.8704
     19        0.6652        0.6753  0.8720
     20        0.6640        0.6739  0.8665
     21        0.6629        0.6

    132        0.5837        0.5946  0.8697
    133        0.5830        0.5940  0.8742
    134        0.5823        0.5935  0.8719
    135        0.5816        0.5929  0.8733
    136        0.5809        0.5923  0.8712
    137        0.5802        0.5918  0.8734
    138        0.5795        0.5912  0.8691
    139        0.5788        0.5906  0.8705
    140        0.5781        0.5901  0.8793
    141        0.5774        0.5895  0.8698
    142        0.5767        0.5890  0.8908
    143        0.5759        0.5884  0.8694
    144        0.5752        0.5878  0.8723
    145        0.5745        0.5873  0.8725
    146        0.5738        0.5867  0.8786
    147        0.5731        0.5862  0.8816
    148        0.5723        0.5856  0.8730
    149        0.5716        0.5851  0.8774
    150        0.5709        0.5845  0.8789
    151        0.5702        0.5840  0.8731
    152        0.5694        0.5834  0.8746
    153        0.5687        0.5829  0.8800
    154        0.5680        0.5

In [ ]:
%%capture output4
from sklearn.model_selection import GridSearchCV


params = {
    'lr': [0.000001, 0.000002,0.000004,0.000008],
    'max_epochs': [50,100,150, 200,250],
}
gs = GridSearchCV(model, params, refit=False, cv=3,n_jobs=1)

In [ ]:
gs.fit(x, y)

In [ ]:
gs

In [ ]:
gs